In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib
import requests
from PyPDF2 import PdfMerger
from urllib.parse import urlparse
from io import BytesIO
from enum import Enum
import re
from urllib.error import URLError, HTTPError  # Import these classes from urllib.error
import ssl
import pymupdf
import os

## Extract Link from 150 Good research paper

In [ ]:
# Read in the csv file
## This is a csv file that teammates manually inspected as good research pape
good_df = pd.read_csv("../data/150_research_papers.csv")[['Link']]
#Add the label -> 1 for relevent
good_df['label'] = 1

In [5]:
good_df.head()
# good_df.shape

,Link,label
0,https://www.science.org/doi/10.1126/science.ad...,1
1,https://www.nature.com/articles/s41566-019-0398-2,1
2,https://www.nature.com/articles/s41560-020-007...,1
3,https://www.science.org/doi/10.1126/science.ab...,1
4,https://www.nature.com/articles/s41467-021-236...,1


## Import bad paper

In [7]:
bad_df = pd.read_csv("../data/irrelevant_papers.txt")
bad_df.head()

,link
0,https://doi.org/10.1016/0021-9797(77)90150-3
1,https://www.sciencedirect.com/science/article/...
2,https://www.sciencedirect.com/science/article/...
3,https://doi.org/10.1107%2FS0021889813019535
4,https://www.sciencedirect.com/science/article/...


In [ ]:
#Add the label -> 0 for irrelevent
bad_df['label'] = 0
# sampled_bad_df.to_csv('bad_paper_small.csv', index=False)

NameError: name 'bad_df' is not defined

In [9]:
bad_df

,link,label
0,https://doi.org/10.1016/0021-9797(77)90150-3,0
1,https://www.sciencedirect.com/science/article/...,0
2,https://www.sciencedirect.com/science/article/...,0
3,https://doi.org/10.1107%2FS0021889813019535,0
4,https://www.sciencedirect.com/science/article/...,0
...,...,...
310,https://www.sciencedirect.com/science/article/...,0
311,https://www.sciencedirect.com/science/article/...,0
312,https://www.sciencedirect.com/science/article/...,0
313,https://doi.org/10.1063/1.1404988,0


# Scraping Text

In [10]:
def get_text(soup):
    #Given a beautiful soup object, it will extact the text
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [ ]:
def create_txt_list(df, col_name):
    def fetch_page(link):
        try:
            # Try accessing the page normally
            return urlopen(link)
        except HTTPError as e:
            if e.code == 403:
                print(f"403 Forbidden error; retrying with headers for {link}")
                hdr = {'User-Agent': 'Mozilla/5.0'}
                request = Request(link, headers=hdr)
                return urlopen(request)
            else:
                raise e  # Re-raise if it’s a different HTTP error
        except (URLError, ssl.SSLError, ValueError) as e:
            print(f"Error accessing {link}: {e}")
            return None

    text_lst = []
    for count, link in enumerate(df[col_name], start=1):
        print(f"Analyzing {count}th link")
        try:
            page = fetch_page(link)
            if page:
                html_content = page.read().decode("utf-8")
                soup = BeautifulSoup(html_content, "html.parser")
                text_lst.append(get_text(soup))
            else:
                text_lst.append("Error, No Access")
        except Exception as e:
            print(f"Unexpected error for {link}: {e}")
            text_lst.append("Unexpected Error No Access")
        
    df['text'] = text_lst
    return df

In [ ]:
#Extract text from bad text dataframe
bad_df_text = create_txt_list(bad_df, "link")

Analyzing 1th link
Page accessed successfully.

Analyzing 2th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 3th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 4th link
Page accessed successfully.

Analyzing 5th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 6th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Page accessed successfully with headers.

Analyzing 7th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 8th link
Page accessed successfully.

Analyzing 9th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 10th link
Page accessed successfully.

In [14]:
bad_df_text.to_csv('../data/text_bad_paper.csv', index=False)

In [15]:
#Extract text from good dataframe
good_df_text = create_txt_list(good_df, "Link")

Analyzing 1th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 2th link
Page accessed successfully.

Analyzing 3th link
Page accessed successfully.

Analyzing 4th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 5th link
Page accessed successfully.

Analyzing 6th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 7th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 8th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 9th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 10th link
HTTP error occurred: 403 -

In [16]:
good_df_text.to_csv('../data/text_good_paper.csv', index=False)

## Head of good_df_text and bad_df_text

In [17]:
good_df_text

,Link,label,text
0,https://www.science.org/doi/10.1126/science.ad...,1,"HTTP Error, No Access"
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
3,https://www.science.org/doi/10.1126/science.ab...,1,"HTTP Error, No Access"
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
...,...,...,...
146,https://doi.org/10.1002%2Fsmll.201803350,1,"HTTP Error, No Access"
147,https://doi.org/10.1038%2Fs41467-022-34203-x,1,Overcoming C60-induced interfacial recombinati...
148,https://doi.org/10.1038%2Fs41560-023-01249-0,1,Bifunctional hole-shuttle molecule for improve...
149,https://doi.org/10.1002%2Fadma.201907757,1,"HTTP Error, No Access"


In [18]:
bad_df_text

,link,label,text
0,https://doi.org/10.1016/0021-9797(77)90150-3,0,Redirecting
1,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
2,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
3,https://doi.org/10.1107%2FS0021889813019535,0,(IUCr) X-ray analysis of residual stress gradi...
4,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
...,...,...,...
310,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
311,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
312,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
313,https://doi.org/10.1063/1.1404988,0,The charge of glass and silica surfaces | The ...


## Filter out the non accessible texts

In [19]:
pass_bad_df = bad_df_text[(bad_df_text['text'] != "HTTP Error, No Access") & (bad_df_text['text'] != "Redirecting")]
pass_bad_df

,link,label,text
3,https://doi.org/10.1107%2FS0021889813019535,0,(IUCr) X-ray analysis of residual stress gradi...
5,https://doi.org/10.1063%2F1.4864778,0,Unusual defect physics in CH3NH3PbI3 perovskit...
9,https://doi.org/10.1039/D3TA00873H,0,Molecular interaction modulating Ruddlesden–Po...
12,https://doi.org/10.1039/C5CE01014D,0,Fundamental growth principles of colloidal met...
14,https://doi.org/10.1038%2Fnatrevmats.2018.17,0,Scalable fabrication of perovskite solar cells...
...,...,...,...
290,https://doi.org/10.1017/9781108394826.004,0,Methods of Colloidal Simulation (Chapter 3) - ...
292,https://doi.org/10.1146/annurev.ms.12.080182.0...,0,Transition Metal Oxide Gels and Colloids | Ann...
300,https://doi.org/10.1039/DC9786500007,0,"The First Rideal Lecture. Microemulsions, a fi..."
302,https://doi.org/10.1038%2Fnphoton.2012.285,0,Strain-engineered artificial atom as a broad-s...


In [20]:
pass_good_df = good_df_text[(good_df_text['text'] != "HTTP Error, No Access") & (good_df_text['text'] != "Redirecting")]
pass_good_df = pass_good_df.rename(columns={'Link': 'link'})
pass_good_df

,link,label,text
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
10,https://doi.org/10.1038%2Fs41586-022-04604-5,1,Stability-limiting heterointerfaces of perovsk...
15,https://doi.org/10.1038%2Fs41467-022-30426-0,1,Imaging and quantifying non-radiative losses a...
...,...,...,...
141,https://doi.org/10.1038%2Fs41586-023-06637-w,1,Anion–π interactions suppress phase impurities...
145,https://doi.org/10.1039%2FD2EE00288D,1,Optimized carrier extraction at interfaces for...
147,https://doi.org/10.1038%2Fs41467-022-34203-x,1,Overcoming C60-induced interfacial recombinati...
148,https://doi.org/10.1038%2Fs41560-023-01249-0,1,Bifunctional hole-shuttle molecule for improve...


## Merge two df together

In [21]:
merged_df = pass_good_df.append(pass_bad_df, ignore_index=True)
merged_df

C:\Users\nicco\AppData\Local\Temp\ipykernel_16452\2237511184.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = pass_good_df.append(pass_bad_df, ignore_index=True)


,link,label,text
0,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
1,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
2,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
3,https://doi.org/10.1038%2Fs41586-022-04604-5,1,Stability-limiting heterointerfaces of perovsk...
4,https://doi.org/10.1038%2Fs41467-022-30426-0,1,Imaging and quantifying non-radiative losses a...
...,...,...,...
127,https://doi.org/10.1017/9781108394826.004,0,Methods of Colloidal Simulation (Chapter 3) - ...
128,https://doi.org/10.1146/annurev.ms.12.080182.0...,0,Transition Metal Oxide Gels and Colloids | Ann...
129,https://doi.org/10.1039/DC9786500007,0,"The First Rideal Lecture. Microemulsions, a fi..."
130,https://doi.org/10.1038%2Fnphoton.2012.285,0,Strain-engineered artificial atom as a broad-s...


In [ ]:
merged_df.to_csv('../merged_label.csv', index=False)